In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
file_name='2020年06月25日周四'

with open ('./{}.txt'.format(file_name)) as f:
    text=f.read()
    f.close()


In [3]:
fenlei={'短融':[],
        '中票':[],
        '企业债':[],
        '其他':[]}


for i,line in enumerate(text.split('\n')):
    line=re.split(' +|\t+',line.strip())
    if len(line)==1 and line!=['']:
        if re.search('|'.join(['短融','中票','企业债','其他']) ,line[0])!=None:
            now_list=re.search('|'.join(['短融','中票','企业债','其他']) ,line[0]).group()
            
    
    if len (line)>=5:
        year=line[0]
        try:
#             rating=line[['AAA|(A\-1)' in i for i in line ].index(True)] # 找评级
            rating=line[[re.search('AAA|(A\-1)',i)!=None  for i in line].index(True)] #找评级

            ytm=line[[re.search('\d+\.\d+|\d+',i)!=None  for i in line[3:]].index(True)+3].replace('%','') #找ytm
            name=line[[re.search('[^\x00-\xff]+',i)!=None  for i in line[1:]].index(True)+1]#找名称
            fenlei[now_list].append([year,name,ytm,rating])

        except:
            pass
            if now_list=='短融':
                print('无法获取信息',line)
            
        
    else:
        pass
        

len(fenlei['短融'])

无法获取信息 ['68D(休1)', '19凯盛科技CP001', '041900342', '', '3.20', 'AA+']
无法获取信息 ['33D(休1)', '19海安开投SCP002', '011902529', '', '4.40', 'AA']
无法获取信息 ['173D(休1)', '19阜阳投资CP001', '041900467', '', '2.85', 'AA+']
无法获取信息 ['115D', '20开封基建SCP001', '012000235', '', '3.63', 'AA']
无法获取信息 ['172D(休2)', '20大足国资SCP001', '012000903', '', '3.82', 'AA']


0

In [4]:
pd.DataFrame(fenlei['中票'])[pd.DataFrame(fenlei['中票'])[1]=='16神华MTN002']

,0,1,2,3
13,2.68Y,16神华MTN002,3.08,AAA
41,2.68Y,16神华MTN002,3.08,AAA
152,2.68Y,16神华MTN002,3.08,AAA


In [5]:
fenlei_df ={i:pd.DataFrame(fenlei[i],columns='剩余期限 简称 收益率 评级'.split(' ')) for  i in fenlei.keys()}
fenlei_df['中票']

,剩余期限,简称,收益率,评级
0,361D,14山西交投MTN001,3.25,AAA/AA+
1,361D,14山西交投MTN001,3.25,AAA/AA+
2,363D,18电网MTN001,2.62,AAA
3,1.12Y,18汇金MTN010,2.65,AAA
4,1.12Y,18汇金MTN010,2.65,AAA
...,...,...,...,...
186,1.34Y,18华电租赁MTN001,3.48,AAA
187,4.85Y,20溧水经开MTN002,4.30,AA+/AAA
188,4.83Y,20甘交建MTN001BC(品种二),4.30,AAA
189,1.07Y,18汇金MTN009,2.60,AAA


In [6]:
fenlei_df['中票'][fenlei_df['中票']['简称']=='16神华MTN002']

,剩余期限,简称,收益率,评级
13,2.68Y,16神华MTN002,3.08,AAA
41,2.68Y,16神华MTN002,3.08,AAA
152,2.68Y,16神华MTN002,3.08,AAA


## 计算天数，排序

In [7]:
def get_day(days:str):
    main_day=days.split('+')[0]
    if 'D' in main_day:
        real_day=float(re.findall('\d+\.\d+|\d+',main_day)[0])
    elif 'Y' in main_day:
        real_day=float(re.findall('\d+\.\d+|\d+',main_day)[0])*365
    else:
        real_day=main_day
    return real_day

In [8]:
for key in fenlei_df:
    df=fenlei_df[key]
    df['day']=df['剩余期限'].apply(lambda x:get_day(x))
    df.sort_values('day',inplace=True)
    df.index=[i for i in range(len(df))]
    fenlei_df[key]=df.drop('day',axis=1) #去掉辅助列
    
fenlei_df['中票'].head()

,剩余期限,简称,收益率,评级
0,308D,18中金集MTN001BC,2.9,AAA
1,308D,18中金集MTN001BC,2.9,AAA
2,310D+NY,16甬城投MTN001,3.42,AAA
3,361D,14山西交投MTN001,3.25,AAA/AA+
4,361D,14山西交投MTN001,3.25,AAA/AA+


In [9]:
# writer = pd.ExcelWriter('信用债成交记录.xlsx')
# for one in fenlei_df:
#     fenlei_df[one].to_excel(writer, sheet_name=one)
# writer.save()

In [10]:
# import os
# os.startfile('信用债成交记录.xlsx')


In [11]:
from shutil import copyfile
copyfile('./页面模板.xlsx','./{}.xlsx'.format(file_name))

'./2020年06月25日周四.xlsx'

## xlwings写入excel


In [14]:
import xlwings as xw
wb=xw.Book('{}.xlsx'.format(file_name))
sht=wb.sheets['Sheet1']
sht


<Book [2020年06月25日周四.xlsx]>

<Sheet [2020年06月25日周四.xlsx]Sheet1>

## 开始分别写入

In [16]:

def get_excel_coor(now_line):
    page=int(now_line/120)
    line=now_line%60
    if now_line%120<60:
        col='A'
    else:
        col='E'
    row=page*62+line+3
    return str(col)+str(row)
now_line=120
print(get_excel_coor(now_line))


A65


In [17]:
def small_title(coor,title):
    end_coor_dic={'A':'D','E':'H'}
    hebing=coor+':'+end_coor_dic[coor[0]]+coor[1:]
    sht.range(coor).value=title
    print(hebing,title)
    sht.range(hebing).api.merge()
    sht.range(hebing).api.HorizontalAlignment = -4108    # -4108 水平居中。 -4131 靠左，-4152 靠右。
small_title('A3','短融')

A3:D3 短融


In [18]:
# 取消合并所有单元格

for page in range(6):
    page_range='A{}:H{}'.format(page*62+3,page*62+62)
    sht.range(page_range).api.unmerge()
    # Borders(11) 内部垂直边线。    # Borders(12) 内部水平边线。

    for line in [11,12]:
        sht.range(page_range).api.Borders(line).LineStyle = 1
        sht.range(page_range).api.Borders(line).Weight = 2
        sht.range(page_range).api.HorizontalAlignment = -4131
        sht.range(page_range).value=None
        sht.range(page_range).api.NumberFormat = "@"


In [19]:
pd.set_option('display.max_rows', None)

pd.set_option('display.max_columns', None)
fenlei_df['中票'].head()


,剩余期限,简称,收益率,评级
0,308D,18中金集MTN001BC,2.9,AAA
1,308D,18中金集MTN001BC,2.9,AAA
2,310D+NY,16甬城投MTN001,3.42,AAA
3,361D,14山西交投MTN001,3.25,AAA/AA+
4,361D,14山西交投MTN001,3.25,AAA/AA+


In [20]:
sht.range('E1').value=file_name


In [21]:
now_line=0


def xiugai_df(write_df):
    buf=write_df.iloc[0,0]
    write_df.set_index('剩余期限',inplace=True)
    write_df.columns=write_df.iloc[0]
    write_df.index.name=buf
    write_df=write_df.iloc[1:] # 通过一个很愚蠢的方式去掉表头
    return write_df

for one in fenlei_df:
    # 输入标签
    coor=get_excel_coor(now_line)
    small_title(coor,one)
    now_line+=1
    
    
    if len(fenlei_df[one])==0:
        continue
    
    line_left=60-now_line%60# 剩下多少行的空间
    small_df=fenlei_df[one]
    
    while len(small_df)>0:
#         print('还剩多少行:',line_left,'需要多少行:',len(small_df))
        coor=get_excel_coor(now_line) # 计算新的坐标位置
        print(coor)
        # 填充剩余位置
        write_df=small_df.iloc[:line_left]
        write_df=xiugai_df(write_df)
        sht.range(coor).value=write_df
        
        # 更新位置
        now_line+=len(write_df)+1
        
        small_df=small_df.iloc[line_left:]
        line_left=60-now_line%60  # 更新剩下多少行的空间
        
#     print('还剩多少行',line_left,'需要多少行',len(small_df))
#     write_df=small_df
#     write_df=xiugai_df(write_df)    
#     sht.range(coor).value=write_df
#     now_line+=line_left


A3:D3 短融
A4:D4 中票
A5
E3
A65
E65
E78:H78 企业债
E79
A127
E127
A189
A247:D247 其他
A248
E189
A251


In [23]:
# write_df=small_df.iloc[:10]
# buf=write_df.iloc[0,0]
# write_df.set_index('剩余期限',inplace=True)
# write_df.columns=write_df.iloc[0]
# write_df.index.name=buf
# write_df=write_df.iloc[1:] # 通过一个很愚蠢的方式去掉表头

# write_df

In [24]:
# sht.range('A10').value=write_df


In [28]:
file_path=r'D:\Desktop\bond_remind_tkinter.exe'
file_name=re.split(r'\|/|\\|//',file_path)[-1].split('.')[0]
file_name

'bond_remind_tkinter'

In [30]:
import openpyxl
